### `<CAW> Studios` Backend Dev Assignment


##### The below codes are mainly used for data population in mongoDB collections

There are two files namely:
- TheatresCSV.csv
- MovieCSV.csv<br/>

which are used to populate the data in mongoDB theatre collection and Movies collection

In [3]:
import csv
import re
import datetime 
from random import randint, choice
from pymongo import MongoClient

In [ ]:
client = MongoClient("mongodb+srv://YOUR MONGO SRV CONNECTION STRING")
db = client.bookmyshow

'''
DATABASE: bookmyshow

# COLLECTIONS:
- user
- movie
- theatre
- screen
- show
- seating
- booking
- city
- session_info

'''

##### Populate city collection


In [ ]:
def populate_cities(db):
    city_col = db.city
    lst = ["Bengaluru","Hyderabad","Mumbai","Chennai","Kolkata","Pune","Delhi","Ahmedabad","Chandigarh","Jaipur"]
    for city in lst:
        ins_id = city_col.insert_one({"name":city})
        print(ins_id)

##### Populate theatre collection


In [ ]:
def populate_theatres(db):
    col = db.theatre
    
    with open('./data/TheatresCSV.csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            address = row["address"].replace('"',"")
            theatre = {"name":row["name"],"address":address,"contact":row["contact"],"city":row["city"]}
            ID = col.insert_one(theatre)
            print(ID)

populate_theatres(db)

##### Populate movie collection


In [ ]:
def populate_movies(db):
    movie_col = db.movie
    
    with open('./data/MovieCSV.csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            genre = row["genre"].split("|")
            cast = row["cast"].split("|")
            movie = {"name":row["name"], "language":row["language"], "genre":genre, "cast":cast, "director":row["director"],"release_date":row["release_date"],"duration":eval(row["duration"])}
            movie_col.insert_one(movie)


populate_movies(db)

In [ ]:
''' USERs are manually created using signup API'''

# def populate_users(db, mflixdb):
#     user_col = db.user
#     user_mflix = mflixdb.users
    
#     cnt = 0
#     for user in user_mflix.find():
#         if(cnt == 10):
#             break
#         user.pop("_id")
#         ID = user_col.insert_one(user)
#         print(ID)
#         cnt+=1

# populate_users(db, mflixdb)



##### Populate screen collection


In [ ]:
# Randomly generate between (1,3) screens for each theatre

def populate_screens(db):
    screen_col = db.screen
    theatre_col = db.theatre
    
    for theatre in theatre_col.find():
        t_id = theatre["_id"]
        no_screens = randint(1,3)
        for num in range(1,no_screens+1):
            screen = {"theatre_id":t_id, "name":"SCR-"+str(num)}
            screen_col.insert_one(screen)
        theatre_col.update({"_id":t_id},{"$set":{"no_screens":no_screens}})
    
    

populate_screens(db)

##### Populate shows collection


In [ ]:
## CHOOSE THE MOVIES WITH REGIONAL LANGUAGE
## to do : create a language preference to choose movies to show


def populate_shows(db):
    movie_col = db.movie
    theatre_col = db.theatre
    screen_col = db.screen
    show_col = db.show
    city_col = db.city
    
    recent_movies = []
    
    # CHANGE REGEX THIS TO THIS MONTH AND LAST MONTH
    d = datetime.date.today().month

    this_month = datetime.date(2020,d.month,1).strftime('%b')
    last_month = datetime.date(2020,d.month-1,1).strftime('%b')
    regexp = ".*-"+last_month+"-20"+"|.*-"+this_month+"-20"
    for movie in movie_col.find({"release_date":{"$regex":regexp}}):
        recent_movies.append(movie)
        
    if(recent_movies == []):return
    
    for theatre in theatre_col.find():
        no_movies_to_show = randint(2,4)
        movies_list = recent_movies[:]
      
        movies_show = [] # FINAL LIST OF MOVIES TO SCREEN
        while(no_movies_to_show):
            movie = choice(movies_list)
            if(movie not in movies_show):
                movies_show.append(movie)
                no_movies_to_show-=1
        
        
        
#         TO GIVE FIRST PRIORITY TO MOVIES IN REGIONAL LANGUAGES
#         regional_movies = []        
#         regional_language = city_col.find_one({"_id":theatre["city_id"]})["regional_language"]
#         for movie in recent_movies:
#             if movie["language"] == regional_language:
#                 regional_movies.append(movie)
#                 movies_list.remove(movie)
#                 break  ## Adding one regional movie to screen
        
        
#         if(regional_movies):
#             if(len(regional_movies)<= no_movies_to_show):
#                 movies_show+=regional_movies
#                 left = no_movies_to_show - len(regional_movies)
        
#         if(movies_list!=[]):
#             for i in range(0,left):
#                 random_movie = choice(movies_list)
#                 movies_show.append(random_movie)
#                 movies_list.remove(random_movie)
            
        no_screens = theatre["no_screens"]
        
        if(no_screens == 1):
            timings = {"SCR-1":["09:30 AM","01:10 PM","04:50 PM","08:15 PM"]}
        elif(no_screens == 2):
            timings = {"SCR-1":["09:30 AM","12:15 PM","03:30 PM","06:45 PM","10:00 PM"],
                       "SCR-2":["10:15 PM","01:10 PM","04:40 PM","07:45 PM"]}
        elif(no_screens == 3):
            timings = {"SCR-1":["09:30 AM","12:45 PM","03:55 PM","07:15 PM"],
                       "SCR-2":["08:45 AM","12:15 PM","04:00 PM","07:10 PM"],
                       "SCR-3":["09:00 AM","12:05 PM","03:30 PM","05:50 PM","10:15 PM"]}
            
        
        
        
        ## CREATE SHOWS FOR THE NEXT ONE WEEKS ##
                
        today = datetime.date.today()
        
               
        no_days = 7   ## No of days from today for which shows are created
        
        for day in range(1,no_days+1):
            for screen in screen_col.find({"theatre_id":theatre["_id"]}):
                times = timings[screen["name"]]
                for time in times:
                    #print("MOVIES SELECTED =", movies_show)
                    movie = choice(movies_show)#RANDOMLY CHOOSE FROM MOVIES_SHOW
                    show = {"show_time":time,"date":str(today + datetime.timedelta(days = day)),"screen_id":screen["_id"],"movie_id":movie["_id"],"theatre_id":theatre["_id"]}
                    show_col.insert_one(show)
                    
        current_movie_ids = [ movie['_id'] for movie in movies_show ]
        theatre_col.update_one({"_id":theatre["_id"]},{"$set":{"currently_playing":current_movie_ids}})       
                
                
populate_shows(db)

##### Populate seating collection


In [ ]:
'''
ASSUMING : Total Number of Seats per Screen is 150 where 

lounge = 20 seats   [Rows = N, O]
platinum = 50 seats [Rows = I,J,K,L,M]
gold = 50 seats     [Rows = D,E,F,G,H]
dress_circle = 30   [Rows = A,B,C]
Each row consists of 10 seats

'''

def get_seats_config():
    rows = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O']
    seats = {"dress_circle":{}, "gold":{}, "platinum":{}, "lounge":{}}
    
    
    for ind in range(0,len(rows)):
        each_row = {}
        for i in range(1,11):
            if(ind<=2):
                seats["dress_circle"][rows[ind]+str(i)] = 0
            elif (ind>2 and ind<8):
                seats["gold"][rows[ind]+str(i)] = 0
            elif (ind>=8 and ind<13):
                seats["platinum"][rows[ind]+str(i)] = 0
            else:
                seats["lounge"][rows[ind]+str(i)] = 0
    
    return seats

def populate_seats(db):
    seat_col = db.seating
    show_col = db.show
    
    for show in show_col.find():
        seats_config = get_seats_config()
        seats = {"seats":seats_config,"show_id":show["_id"]}
        seat_col.insert_one(seats)
    
                
populate_seats(db)